#### 2022 Spring Semester
##### Sungwon Chae, Undergraduate
##### dept. of Industrial Engineering

In [ ]:
US patent data

#### 1) data preprocessing

In [1]:
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_corpus, common_dictionary

import pandas as pd
import numpy as np
import spacy
import re
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
import warnings
warnings.filterwarnings('ignore')
    
    
directory = '/Users/chaesungwon/Desktop/SNU SDE/'
file = 'CSVAll2020.csv'
data = pd.read_csv(directory+file)

nlp = spacy.load("en_core_web_sm")
    
source = 'abstract'
docs = data[source].tolist()
docs = [nlp(i) for i in docs]
docs =[[token.lemma_ for token in doc] for doc in docs]
docs = [[token for token in doc if len(token) > 2] for doc in docs] 
docs =[[re.sub(r"[^a-zA-Z0-9-]","",token) for token in doc ] for doc in docs] #특수문자 교체
docs =[[token for token in doc if not token.isdigit() ] for doc in docs]  #숫자 제거 
docs = [[token for token in doc if len(token) > 2] for doc in docs] # 길이기반 제거
with open(directory + 'stopwords_uspto.txt') as f:
    stopwords = [line.rstrip() for line in f]
    
docs =[[token for token in doc if token not in stopwords] for doc in docs] #스탑워즈 제거

#### 2) data tuning

In [2]:
# -*- coding: utf-8 -*-

from gensim import models

# from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
import pandas as pd
import numpy as np


def compute_coherence_values(corpus, dictionary, texts, k, a, b, method = "u_mass"):
    #c_v is most frequently used measure for computing the coherence
   
    result = {}
   
    lda_model = models.LdaModel(corpus=corpus,
                             id2word=dictionary,
                             num_topics=k,
                             random_state=12345,
                             # chunksize=100,
                             passes=10,
                             alpha=0.01,      #변경값
                             eta=1,           #변경값
                             )
   
    result['perplexity'] = lda_model.log_perplexity(corpus)
   
    # for method in ['u_mass', 'c_v', 'c_uci', 'c_npmi'] :
    for method in ['u_mass'] :
        coherence_model_lda = CoherenceModel(model=lda_model,
                                             texts=texts,
                                             dictionary=dictionary,
                                             coherence= method)
        result[method] = coherence_model_lda.get_coherence()
       
    return result


def tunning_ab(texts, dct, corpus, k) :
   
    grid = {}
    grid['Validation_Set'] = {}
   
    # Alpha parameter
    alpha = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
    # # alpha.append('symmetric')
    # # alpha.append('asymmetric')
   
    # # Beta parameter
    # # beta = list(np.arange(0.01, 1, 0.3))
    beta = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
    # beta.append('symmetric')

    # Validation sets
    corpus_sets = [corpus]
   
    corpus_title = ['100% Corpus']
   
    model_results = {'Validation_Set': [],
                     'Topics': [],
                     'Alpha': [],
                     'Beta': [],
                     'Perplexity': [],
                     'U_mass' : [],
                     # 'C_v' : [],
                     # 'C_uci' : [],
                     # 'C_npmi' : [],
                     
                    }
    # Can take a long time to run
    if 1 == 1:
       
        cnt = 0
       
        # iterate through validation corpuses
        for i in range(len(corpus_sets)):
            # iterate through number of topics
            # for k in topics_range:
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        result = compute_coherence_values(corpus=corpus_sets[i],
                                                      dictionary=dct,
                                                      texts = texts,
                                                      k=k,
                                                      a=a,
                                                      b=b)
               
                       
                        # Save the model results
                        model_results['Validation_Set'].append(corpus_title[i])
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Beta'].append(b)
                        model_results['Perplexity'].append(result['perplexity'])
                        model_results['U_mass'].append(result['u_mass'])
                        # model_results['C_v'].append(result['c_v'])
                        # model_results['C_uci'].append(result['c_uci'])
                        # model_results['C_npmi'].append(result['c_npmi'])
                       
                       
                        cnt +=1
                        print("전체 {} 중에서 {} ".format(len(alpha) *len(beta),cnt))
                       
    return(pd.DataFrame(model_results))


#temp_data = data_.loc[data_['platform'] == 'Amazon',:].reset_index(drop = 1)
temp_data = data.loc[:,:].reset_index(drop = 1)
temp_data['docs'] = docs   #KeyError evacuation
# temp_data = temp_data.sample(n= int(len(temp_data)), random_state = 1234).reset_index(drop = 1)
texts =  temp_data['docs'] 
   
       
common_dictionary = Dictionary(texts)
common_dictionary.filter_extremes(no_below = 10)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

from gensim import models

       
# tunning
# tunning_result = tunning_gs(texts, common_dictionary, common_corpus, 10, 41, 2)
tunning_result = tunning_ab(texts, common_dictionary, common_corpus, 20)

전체 49 중에서 1 
전체 49 중에서 2 
전체 49 중에서 3 
전체 49 중에서 4 
전체 49 중에서 5 
전체 49 중에서 6 
전체 49 중에서 7 
전체 49 중에서 8 
전체 49 중에서 9 
전체 49 중에서 10 
전체 49 중에서 11 
전체 49 중에서 12 
전체 49 중에서 13 
전체 49 중에서 14 
전체 49 중에서 15 
전체 49 중에서 16 
전체 49 중에서 17 
전체 49 중에서 18 
전체 49 중에서 19 
전체 49 중에서 20 
전체 49 중에서 21 
전체 49 중에서 22 
전체 49 중에서 23 
전체 49 중에서 24 
전체 49 중에서 25 
전체 49 중에서 26 
전체 49 중에서 27 
전체 49 중에서 28 
전체 49 중에서 29 
전체 49 중에서 30 
전체 49 중에서 31 
전체 49 중에서 32 
전체 49 중에서 33 
전체 49 중에서 34 
전체 49 중에서 35 
전체 49 중에서 36 
전체 49 중에서 37 
전체 49 중에서 38 
전체 49 중에서 39 
전체 49 중에서 40 
전체 49 중에서 41 
전체 49 중에서 42 
전체 49 중에서 43 
전체 49 중에서 44 
전체 49 중에서 45 
전체 49 중에서 46 
전체 49 중에서 47 
전체 49 중에서 48 
전체 49 중에서 49 


#### 3) LDA modeling

In [3]:
texts = docs

dct = Dictionary(texts)
dct.filter_extremes(no_below = 5,
                    no_above=0.1)
corpus = [dct.doc2bow(text) for text in texts]

In [4]:
#%% 4. LDA  #수정

lda_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         num_topics=6, #변경
                         random_state=100,
                         chunksize=100,
                         passes=10,
                         alpha=0.01,   #변경
                         eta=1,        #변경
                         # workers=15)
                         )

#%% 응용 1

topics = lda_model.print_topics(num_words=5)

for topic in topics:
    print(topic)

#%% 응용 2

for idx, topic_list in enumerate(lda_model[corpus]):
    # if idx==5: break
    print(idx,'번째 문서의 topic 비율은',topic_list)

#%% 응용 3 https://wikidocs.net/30708

def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

topictable = make_topictable_per_doc(lda_model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

topictable.to_csv(directory + 'topictable.csv', index = 0 , encoding = "euc-kr")

#%% 응용 4

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, dct)
pyLDAvis.save_html(lda_viz, directory + 'lda_viz_All2020.html')

(0, '0.045*"touch" + 0.040*"sensor" + 0.026*"antenna" + 0.025*"sense" + 0.018*"sensing"')
(1, '0.019*"network" + 0.016*"service" + 0.016*"wireless" + 0.014*"message" + 0.013*"request"')
(2, '0.038*"voltage" + 0.028*"power" + 0.019*"block" + 0.017*"value" + 0.016*"operation"')
(3, '0.016*"content" + 0.014*"object" + 0.013*"can" + 0.012*"application" + 0.010*"audio"')
(4, '0.045*"pixel" + 0.035*"pattern" + 0.029*"semiconductor" + 0.027*"gate" + 0.025*"pad"')
(5, '0.025*"housing" + 0.021*"member" + 0.020*"side" + 0.019*"panel" + 0.015*"part"')
0 번째 문서의 topic 비율은 [(3, 0.998811)]
1 번째 문서의 topic 비율은 [(0, 0.16981456), (3, 0.82893765)]
2 번째 문서의 topic 비율은 [(3, 0.9987198)]
3 번째 문서의 topic 비율은 [(0, 0.096107036), (1, 0.13875154), (2, 0.3070595), (5, 0.45771185)]
4 번째 문서의 topic 비율은 [(1, 0.50396806), (3, 0.4948573)]
5 번째 문서의 topic 비율은 [(3, 0.99783164)]
6 번째 문서의 topic 비율은 [(1, 0.5618495), (2, 0.18841864), (3, 0.24933209)]
7 번째 문서의 topic 비율은 [(0, 0.26420802), (3, 0.7349596)]
8 번째 문서의 topic 비율은 [(3, 0.9

244 번째 문서의 topic 비율은 [(1, 0.9979213)]
245 번째 문서의 topic 비율은 [(3, 0.90541714), (5, 0.09223754)]
246 번째 문서의 topic 비율은 [(0, 0.2735032), (2, 0.40538415), (3, 0.06078898), (5, 0.25983638)]
247 번째 문서의 topic 비율은 [(2, 0.7767192), (3, 0.19912823), (4, 0.02362659)]
248 번째 문서의 topic 비율은 [(1, 0.9634237), (3, 0.035951737)]
249 번째 문서의 topic 비율은 [(0, 0.7585464), (4, 0.24045502)]
250 번째 문서의 topic 비율은 [(3, 0.083046734), (5, 0.91615397)]
251 번째 문서의 topic 비율은 [(1, 0.08595652), (3, 0.91244704)]
252 번째 문서의 topic 비율은 [(4, 0.9958539)]
253 번째 문서의 topic 비율은 [(4, 0.48217562), (5, 0.5168501)]
254 번째 문서의 topic 비율은 [(2, 0.9991236)]
255 번째 문서의 topic 비율은 [(0, 0.27150974), (4, 0.72756094)]
256 번째 문서의 topic 비율은 [(0, 0.78115606), (5, 0.21779278)]
257 번째 문서의 topic 비율은 [(5, 0.99891424)]
258 번째 문서의 topic 비율은 [(1, 0.32579768), (2, 0.6733869)]
259 번째 문서의 topic 비율은 [(2, 0.096007735), (5, 0.9033369)]
260 번째 문서의 topic 비율은 [(2, 0.9992313)]
261 번째 문서의 topic 비율은 [(0, 0.14756635), (4, 0.15219341), (5, 0.6994911)]
262 번째 문서의 topic 비

452 번째 문서의 topic 비율은 [(2, 0.9982179)]
453 번째 문서의 topic 비율은 [(4, 0.9993425)]
454 번째 문서의 topic 비율은 [(0, 0.9986133)]
455 번째 문서의 topic 비율은 [(0, 0.8733433), (3, 0.12563244)]
456 번째 문서의 topic 비율은 [(0, 0.08604214), (1, 0.15596254), (2, 0.059277855), (3, 0.47179535), (5, 0.22668433)]
457 번째 문서의 topic 비율은 [(1, 0.09465912), (2, 0.34884328), (3, 0.48316377), (5, 0.07284665)]
458 번째 문서의 topic 비율은 [(2, 0.99762577)]
459 번째 문서의 topic 비율은 [(3, 0.9988109)]
460 번째 문서의 topic 비율은 [(1, 0.6382225), (2, 0.36044645)]
461 번째 문서의 topic 비율은 [(1, 0.55190545), (3, 0.44655946)]
462 번째 문서의 topic 비율은 [(5, 0.9990574)]
463 번째 문서의 topic 비율은 [(0, 0.32108867), (3, 0.6774856)]
464 번째 문서의 topic 비율은 [(3, 0.03713524), (5, 0.9621382)]
465 번째 문서의 topic 비율은 [(1, 0.8826595), (5, 0.11605241)]
466 번째 문서의 topic 비율은 [(1, 0.7726623), (2, 0.22628655)]
467 번째 문서의 topic 비율은 [(0, 0.808034), (4, 0.1912121)]
468 번째 문서의 topic 비율은 [(2, 0.99868625)]
469 번째 문서의 topic 비율은 [(3, 0.998686)]
470 번째 문서의 topic 비율은 [(0, 0.20472454), (4, 0.7935408)]
471

656 번째 문서의 topic 비율은 [(4, 0.15878557), (5, 0.84010476)]
657 번째 문서의 topic 비율은 [(4, 0.4446053), (5, 0.55442)]
658 번째 문서의 topic 비율은 [(0, 0.6980212), (4, 0.30069077)]
659 번째 문서의 topic 비율은 [(1, 0.12775269), (3, 0.87165076)]
660 번째 문서의 topic 비율은 [(3, 0.9991388)]
661 번째 문서의 topic 비율은 [(1, 0.1792454), (2, 0.11202262), (3, 0.7083707)]
662 번째 문서의 topic 비율은 [(0, 0.046641152), (1, 0.6342393), (2, 0.3185201)]
663 번째 문서의 topic 비율은 [(0, 0.5618931), (4, 0.21530604), (5, 0.22199127)]
664 번째 문서의 topic 비율은 [(0, 0.09452666), (4, 0.7921576), (5, 0.11252742)]
665 번째 문서의 topic 비율은 [(0, 0.11383381), (3, 0.8854261)]
666 번째 문서의 topic 비율은 [(1, 0.61173445), (2, 0.3869775)]
667 번째 문서의 topic 비율은 [(0, 0.017912677), (2, 0.98138624)]
668 번째 문서의 topic 비율은 [(2, 0.071363196), (4, 0.92738914)]
669 번째 문서의 topic 비율은 [(0, 0.45644405), (3, 0.05251843), (4, 0.31854767), (5, 0.17201424)]
670 번째 문서의 topic 비율은 [(1, 0.43703532), (2, 0.56209594)]
671 번째 문서의 topic 비율은 [(1, 0.9970685)]
672 번째 문서의 topic 비율은 [(2, 0.997069)]
673 번째 문서의 

885 번째 문서의 topic 비율은 [(1, 0.1279882), (3, 0.51357317), (5, 0.35719135)]
886 번째 문서의 topic 비율은 [(1, 0.9966791)]
887 번째 문서의 topic 비율은 [(3, 0.9988108)]
888 번째 문서의 topic 비율은 [(3, 0.9991237)]
889 번째 문서의 topic 비율은 [(3, 0.6146246), (4, 0.0714429), (5, 0.31340656)]
890 번째 문서의 topic 비율은 [(1, 0.3927605), (3, 0.60664296)]
891 번째 문서의 topic 비율은 [(1, 0.71157795), (2, 0.2342584), (3, 0.053576034)]
892 번째 문서의 topic 비율은 [(0, 0.94125366), (2, 0.05777209)]
893 번째 문서의 topic 비율은 [(0, 0.9097305), (4, 0.08956843)]
894 번째 문서의 topic 비율은 [(4, 0.9981522)]
895 번째 문서의 topic 비율은 [(0, 0.640138), (4, 0.3583834)]
896 번째 문서의 topic 비율은 [(2, 0.2454375), (4, 0.75387347)]
897 번째 문서의 topic 비율은 [(2, 0.098295525), (4, 0.9009909)]
898 번째 문서의 topic 비율은 [(4, 0.99865085)]
899 번째 문서의 topic 비율은 [(3, 0.99861294)]
900 번째 문서의 topic 비율은 [(0, 0.4595647), (4, 0.12441498), (5, 0.415232)]
901 번째 문서의 topic 비율은 [(0, 0.12881942), (5, 0.86944586)]
902 번째 문서의 topic 비율은 [(0, 0.10505814), (3, 0.17544281), (5, 0.7187684)]
903 번째 문서의 topic 비율은 [(5, 

...

6370 번째 문서의 topic 비율은 [(2, 0.17398174), (4, 0.8246875)]
6371 번째 문서의 topic 비율은 [(4, 0.99868613)]
6372 번째 문서의 topic 비율은 [(2, 0.06844773), (4, 0.9303778)]
6373 번째 문서의 topic 비율은 [(0, 0.0435032), (4, 0.95535564)]
6374 번째 문서의 topic 비율은 [(4, 0.5308859), (5, 0.4677375)]
6375 번째 문서의 topic 비율은 [(0, 0.3457522), (4, 0.6534642)]
6376 번째 문서의 topic 비율은 [(4, 0.9988651)]
6377 번째 문서의 topic 비율은 [(0, 0.34008276), (4, 0.6587762)]
6378 번째 문서의 topic 비율은 [(4, 0.9989375)]
6379 번째 문서의 topic 비율은 [(4, 0.99927586)]
6380 번째 문서의 topic 비율은 [(4, 0.99878216)]
6381 번째 문서의 topic 비율은 [(4, 0.9991674)]
6382 번째 문서의 topic 비율은 [(2, 0.51529086), (4, 0.48356813)]
6383 번째 문서의 topic 비율은 [(1, 0.023328587), (4, 0.9756971)]
6384 번째 문서의 topic 비율은 [(4, 0.9982793)]
6385 번째 문서의 topic 비율은 [(4, 0.9989596)]
6386 번째 문서의 topic 비율은 [(4, 0.9989595)]
6387 번째 문서의 topic 비율은 [(2, 0.7980829), (4, 0.20126192)]
6388 번째 문서의 topic 비율은 [(0, 0.13139714), (2, 0.86791384)]
6389 번째 문서의 topic 비율은 [(2, 0.96159935), (4, 0.037687115)]
6390 번째 문서의 topic 비율은 [(2, 

6585 번째 문서의 topic 비율은 [(1, 0.63841885), (2, 0.16288054), (3, 0.03025527), (5, 0.16813304)]
6586 번째 문서의 topic 비율은 [(3, 0.99857366)]
6587 번째 문서의 topic 비율은 [(1, 0.092697226), (3, 0.8624595), (4, 0.04364601)]
6588 번째 문서의 topic 비율은 [(0, 0.123365544), (3, 0.8754244)]
6589 번째 문서의 topic 비율은 [(2, 0.35246328), (3, 0.6463265)]
6590 번째 문서의 topic 비율은 [(3, 0.9544745), (5, 0.0445011)]
6591 번째 문서의 topic 비율은 [(2, 0.11329735), (5, 0.88579476)]
6592 번째 문서의 topic 비율은 [(2, 0.9403342), (4, 0.058999732)]
6593 번째 문서의 topic 비율은 [(1, 0.753023), (2, 0.22208616), (4, 0.024224993)]
6594 번째 문서의 topic 비율은 [(1, 0.47791228), (2, 0.40740865), (3, 0.11427926)]
6595 번째 문서의 topic 비율은 [(1, 0.61949897), (2, 0.37890473)]
6596 번째 문서의 topic 비율은 [(1, 0.34350646), (2, 0.5953883), (5, 0.060613833)]
6597 번째 문서의 topic 비율은 [(1, 0.10156139), (3, 0.8976394)]
6598 번째 문서의 topic 비율은 [(3, 0.83040595), (5, 0.16874403)]
6599 번째 문서의 topic 비율은 [(0, 0.033696488), (3, 0.9651625)]
6600 번째 문서의 topic 비율은 [(0, 0.79787785), (3, 0.20132291)]
6601 번째 

6817 번째 문서의 topic 비율은 [(1, 0.14800611), (2, 0.85124)]
6818 번째 문서의 topic 비율은 [(2, 0.9988902)]
6819 번째 문서의 topic 비율은 [(2, 0.998218)]
6820 번째 문서의 topic 비율은 [(4, 0.40335697), (5, 0.595355)]
6821 번째 문서의 topic 비율은 [(1, 0.34551278), (2, 0.6528246)]
6822 번째 문서의 topic 비율은 [(0, 0.30126143), (4, 0.2562109), (5, 0.44208688)]
6823 번째 문서의 topic 비율은 [(4, 0.3472163), (5, 0.6514957)]
6824 번째 문서의 topic 비율은 [(0, 0.2660089), (4, 0.27609292), (5, 0.45723233)]
6825 번째 문서의 topic 비율은 [(5, 0.9984874)]
6826 번째 문서의 topic 비율은 [(2, 0.06270312), (4, 0.20872341), (5, 0.7279619)]
6827 번째 문서의 topic 비율은 [(4, 0.9986507)]
6828 번째 문서의 topic 비율은 [(4, 0.5435019), (5, 0.45476255)]
6829 번째 문서의 topic 비율은 [(4, 0.99861336)]
6830 번째 문서의 topic 비율은 [(2, 0.12068622), (4, 0.878426)]
6831 번째 문서의 topic 비율은 [(2, 0.25061333), (4, 0.74836254)]
6832 번째 문서의 topic 비율은 [(4, 0.30363482), (5, 0.69488686)]
6833 번째 문서의 topic 비율은 [(4, 0.9954791)]
6834 번째 문서의 topic 비율은 [(4, 0.87189764), (5, 0.12692772)]
6835 번째 문서의 topic 비율은 [(0, 0.22094409), (4, 0

7024 번째 문서의 topic 비율은 [(3, 0.4291776), (4, 0.14699554), (5, 0.42320248)]
7025 번째 문서의 topic 비율은 [(0, 0.12811548), (2, 0.49000606), (3, 0.06522161), (5, 0.31627253)]
7026 번째 문서의 topic 비율은 [(4, 0.26980922), (5, 0.7278456)]
7027 번째 문서의 topic 비율은 [(5, 0.9988903)]
7028 번째 문서의 topic 비율은 [(3, 0.29163542), (5, 0.7076244)]
7029 번째 문서의 topic 비율은 [(5, 0.99865055)]
7030 번째 문서의 topic 비율은 [(5, 0.9992071)]
7031 번째 문서의 topic 비율은 [(0, 0.42672646), (4, 0.19248818), (5, 0.38023037)]
7032 번째 문서의 topic 비율은 [(2, 0.7359549), (4, 0.26299393)]
7033 번째 문서의 topic 비율은 [(1, 0.9988112)]
7034 번째 문서의 topic 비율은 [(1, 0.998389)]
7035 번째 문서의 topic 비율은 [(1, 0.99723136)]
7036 번째 문서의 topic 비율은 [(1, 0.7346497), (2, 0.26476237)]
7037 번째 문서의 topic 비율은 [(1, 0.99833643)]
7038 번째 문서의 topic 비율은 [(1, 0.22686517), (2, 0.22764927), (3, 0.5449503)]
7039 번째 문서의 topic 비율은 [(1, 0.46280032), (2, 0.5358687)]
7040 번째 문서의 topic 비율은 [(1, 0.70424056), (3, 0.29422423)]
7041 번째 문서의 topic 비율은 [(1, 0.9991534)]
7042 번째 문서의 topic 비율은 [(1, 0.9987517)]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/dev

#### appendix